# TurboInsight: Vehicle Sales Analysis

## About the Data

### Context
The dataset contains 3 million real world used cars details.

### Content
This data was obtained by running a self made crawler on Cargurus inventory in September 2020.

### Columns

1- vin: Type String. Vehicle Identification Number is a unique encoded string for every vehicle. Read more at https://www.autocheck.com/vehiclehistory/vin-basics

2- back_legroom: Type String. Legroom in the rear seat.

3- bed: Type String. Category of bed size(open cargo area) in pickup truck. Null usually means the vehicle isn't a pickup truck

4- bed_height: Type String. Height of bed in inches

5- bed_length: Type String. Length of bed in inches

6- body_type: Type String. Body Type of the vehicle. Like Convertible, Hatchback, Sedan, etc.

7- cabin: Type String. Category of cabin size(open cargo area) in pickup truck. Eg: Crew Cab, Extended Cab, etc.

8- city: Type String. city where the car is listed. Eg: Houston, San Antonio, etc.

9- city_fuel_economy: Type Float. Fuel economy in city traffic in km per litre

10- combine_fuel_economy: Type Float. Combined fuel economy is a weighted average of City and Highway fuel economy in km per litre

11- daysonmarket: Type Integer. Days since the vehicle was first listed on the website.

12- dealer_zip: Type Integer. Zipcode of the dealer

13- description: Type String. Vehicle description on the vehicle's listing page

14- engine_cylinders: Type String. The engine configuration. Eg: I4, V6, etc.

15- engine_displacement: Type Float. engine_displacement is the measure of the cylinder volume swept by all of the pistons of a piston engine, excluding the combustion chambers.

16- engine_type: Type String. The engine configuration. Eg: I4, V6, etc.

17- exterior_color: Type String. Exterior color of the vehicle, usually a fancy one same as the brochure.

18- fleet: Type Boolean. Whether the vehicle was previously part of a fleet.

19- frame_damaged: Type Boolean. Whether the vehicle has a damaged frame.

20- franchise_dealer: Type Boolean. Whether the dealer is a franchise dealer.

21- franchise_make: Type String. The company that owns the franchise.

22- front_legroom: Type String. The legroom in inches for the passenger seat

23- fuel_tank_volume: Type String. Fuel tank's filling capacity in gallons

24- fuel_type: Type String. Dominant type of fuel ingested by the vehicle.

25- has_accidents: Type Boolean. Whether the vin has any accidents registered.

26- height: Type String. Height of the vehicle in inches

27- highway_fuel_economy: Type Float. Fuel economy in highway traffic in km per litre

28- horsepower: Type Float. Horsepower is the power produced by an engine.

29- interior_color: Type String. Interior color of the vehicle, usually a fancy one same as the brochure.

30- isCab: Type Boolean. Whether the vehicle was previously taxi/cab.

31- is_certified: Type Boolean. Whether the vehicle is certified. Certified cars are covered through warranty period

32- is_cpo: Type Boolean. Pre-owned cars certified by the dealer. Certified vehicles come with a manufacturer warranty for free repairs for a certain time period. Read more at https://www.cartrade.com/blog/2015/auto-guides/pros-and-cons-of-buying-a-certified-pre-owned-car-1235.html

33- is_new: Type Boolean. If True means the vehicle was launched less than 2 years ago.

34- is_oemcpo: Type Boolean. Pre-owned cars certified by the manufacturer. Read more at https://www.cargurus.com/Cars/articles/know_the_difference_dealership_cpo_vs_manufacturer_cpo

35- latitude: Type Float. Latitude from the geolocation of the dealership.

36- length: Type String. Length of the vehicle in inches

37- listed_date: Type String. The date the vehicle was listed on the website. Does not make days_on_market obsolete. The prices is days_on_market days after the listed date.

38- listing_color: Type String. Dominant color group from the exterior color.

39- listing_id: Unique Type Integer. Listing id from the website

40- longitude: Type Float. Longitude from the geolocation of the dealership.

41- main_picture_url: Type String.

# Importing Libraries

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Data Ingestion

In [3]:
spark = (
    SparkSession
    .builder
    .config('spark.jars', './config/mysql-connector-j-8.3.0.jar')
    .appName('Used Cars ETL')
    .master('local[*]')
    .getOrCreate()
)

In [4]:
#spark.stop()

In [5]:
spark

In [6]:
df = spark.read.option('header',True).csv('data/used_cars_data.csv')

In [7]:
# check partitions number
df.rdd.getNumPartitions()

75

# Data Transformation

In [8]:
print(f'Rows Count:{df.count()}')
print()
print('Data Schema:')
df.printSchema()

Rows Count:3000507

Data Schema:
root
 |-- vin: string (nullable = true)
 |-- back_legroom: string (nullable = true)
 |-- bed: string (nullable = true)
 |-- bed_height: string (nullable = true)
 |-- bed_length: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- cabin: string (nullable = true)
 |-- city: string (nullable = true)
 |-- city_fuel_economy: string (nullable = true)
 |-- combine_fuel_economy: string (nullable = true)
 |-- daysonmarket: string (nullable = true)
 |-- dealer_zip: string (nullable = true)
 |-- description: string (nullable = true)
 |-- engine_cylinders: string (nullable = true)
 |-- engine_displacement: string (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- exterior_color: string (nullable = true)
 |-- fleet: string (nullable = true)
 |-- frame_damaged: string (nullable = true)
 |-- franchise_dealer: string (nullable = true)
 |-- franchise_make: string (nullable = true)
 |-- front_legroom: string (nullable = true)
 |-- fuel_t

In [9]:
# replacing -- with null
df = df.replace('--',None)

In [10]:
# drop duplicates 
df = df.dropDuplicates()

In [11]:
# dropping null columns
df = df.drop('bed_height')
df = df.drop('combine_fuel_economy')
df = df.drop('is_certified')
df = df.drop('cabin')
df = df.drop('vehicle_damage_category')
df = df.drop('bed')
df = df.drop('bed_length')
df = df.drop('main_picture_url')
# I will be remaking this column so I'll be deleing it
df = df.drop('horsepower')

In [12]:
# The string NULL exists, so this replace it with actual nulls.
df = df.replace('NULL',None)

In [13]:
# dropping dirty rows
vin_pattern = r'[a-z]|\.|\s'
df = df.filter(F.length(df['vin'])==17)
df = df.filter((~ df['vin'].rlike(vin_pattern)))
df = df.withColumn('vin',F.regexp_replace('vin','False',''))
df = df.filter(df['vin'].isNotNull())

In [14]:
df = df.withColumn('price',F.col('price').cast('Float'))
df = df.filter(F.col('price').isNotNull())

In [15]:
# cleaning columns and changing data types

# Back Legroom
df = df.withColumn('back_legroom',
                   F.regexp_replace(string='back_legroom',pattern='[^0-9.]',replacement='')) # inches
df = df.withColumn('back_legroom',
                   df.back_legroom.cast('Float'))
df = df.withColumnRenamed('back_legroom',
                          'back_legroom_inches')

# City Fuel Economy
df = df.withColumn('city_fuel_economy',
                   df.city_fuel_economy.cast('Float'))
# Days On Market
df = df.withColumn('daysonmarket',
                   df.daysonmarket.cast('Int'))
# Dealer ZIP
df = df.withColumn('dealer_zip',
                   df.dealer_zip.cast('Int'))

# Engine Cylenders ^.{112,}$
df = df.withColumn('engine_cylinders',
                   F.regexp_replace(string='engine_cylinders',pattern='^.{20,}$',replacement=''))
                   
# Engine Displacement
df = df.withColumn('engine_displacement',
                   df.engine_displacement.cast('Float'))
# Fleet
df = df.withColumn('fleet',
                   df.fleet.cast('Boolean'))
# Frame Damaged
df = df.withColumn('frame_damaged',
                   df.frame_damaged.cast('Boolean'))
# Franchise Dealer
df = df.withColumn('franchise_dealer',
                   df. franchise_dealer.cast('Boolean'))
# Front Legroom
df = df.withColumn('front_legroom',
                   F.regexp_replace(string='front_legroom',pattern='[^0-9.]',replacement='')) # inches
df = df.withColumn('front_legroom',
                   df.front_legroom.cast('Float'))
df = df.withColumnRenamed('front_legroom',
                         'front_legroom_inches')
# Fuel Tank Volume
df = df.withColumn('fuel_tank_volume',
                   F.regexp_replace(string='fuel_tank_volume',pattern='[^0-9.]',replacement='')) # gallons
df = df.withColumn('fuel_tank_volume',
                   df.fuel_tank_volume.cast('Float'))
df = df.withColumnRenamed('fuel_tank_volume',
                          'fuel_tank_volume_gallons')
# Fuel Type
# these are the fuel types I found on the website https://www.cargurus.com/
valid_fuels = ['Gasoline', 'Diesel', 'Biodiesel', 'Flex Fuel Vehicle', 'Hybrid', 'Electric']
# df = df.filter(df.fuel_type.rlike('Gasoline|Diesel|Biodiesel|Flex Fuel Vehicle|Hybrid|Electric'))
df = df.withColumn("fuel_type",
                   F.when(F.col("fuel_type").isin(valid_fuels),
                          F.col("fuel_type")).otherwise(None))
# Has Accidents
df = df.withColumn('has_accidents',
                   df.has_accidents.cast('Boolean'))
# Height
df = df.withColumn('height',
                   F.regexp_replace(string='height',pattern='[^0-9.]',replacement='')) # inches
df = df.withColumn('height',
                   df.height.cast('Float'))
df = df.withColumnRenamed('height',
                          'height_inches')
# Highway Fuel Economy
df = df.withColumn('highway_fuel_economy',
                   df.highway_fuel_economy.cast('Float'))

# interior_color
df = df.withColumn(
    'interior_color',
    F.regexp_replace('interior_color', '^.$', ''))


# Is Cab, Is Certified Pre Owned Car, Is New, Is Pre-Owned Certified By The Manufacturer. 
df = df.withColumn('isCab',
                   df.isCab.cast('Boolean'))
df = df.withColumn('is_cpo',
                   df.is_cpo.cast('Boolean'))
df = df.withColumn('is_new',
                   df.is_new.cast('Boolean'))
df = df.withColumn('is_oemcpo',
                   df.is_oemcpo.cast('Boolean'))

# Latitude & Longitude
df = df.withColumn('latitude',
                   df.latitude.cast('Float'))
df = df.withColumn('longitude',
                   df.longitude.cast('Float'))

# Length ('Inches')
df = df.withColumn('length',
                   F.regexp_replace(string='length',pattern='[^0-9.]',replacement='')) # inches
df = df.withColumn('length',
                   df.length.cast('Float'))
df = df.withColumnRenamed('length',
                          'length_inches')

# Listed Date
df = df.withColumn('listed_date',
                   df.listed_date.cast('Date'))

# Listing Color
df = df.withColumn('listing_color',
                   F.regexp_replace(string='listing_color',pattern='(.*[a-z].*)|(.*\d.*)',replacement=''))
# Listing ID
df = df.withColumn('listing_id',
                   df.listing_id.cast('Int'))

# Major Options
df = df.withColumn(
    "major_options",
    F.when(
        F.regexp_extract(F.col("major_options"), r"^\[(.*)\]$", 1) != "",
        F.split(F.regexp_extract(F.col("major_options"), r"^\[(.*)\]$", 1), ","),
    ).otherwise(F.lit(None)),
)
df = df.withColumn('major_options', F.col('major_options').cast('String'))

# Maximum Seating
df = df.withColumn('maximum_seating',
                   F.regexp_replace(string='maximum_seating',pattern='[^0-9]',replacement=''))\
.withColumn('maximum_seating',
            F.regexp_replace(string='maximum_seating',pattern=r'\b(?:\d{3,}\.\d+|\d{3,}|\d{1,2}\.\d+)\b',replacement=''))
df = df.withColumn('maximum_seating',
                   df.maximum_seating.cast('Int'))

# Mileage
df = df.withColumn('mileage',
                   df.mileage.cast('Float'))

# Owner Count
df = df.withColumn('owner_count',
                   df.owner_count.cast('Int'))

# Power & Torque
lbft_pattern = r"(\d+) lb-ft"
hp_pattern = r"(\d+) hp"
rpm_pattern = r"(\d{1,3},\d{3})"
# # Extract numeric values and RPM for torque
# 'torque_lbft'
df = df.withColumn('torque_lbft', 
                   F.regexp_extract(F.col('torque'), lbft_pattern, 1).cast('Int'))

# 'torque_rpm'
df = df.withColumn('torque_rpm', 
                   F.regexp_extract('torque', rpm_pattern, 1))\
.withColumn('torque_rpm',
            F.regexp_replace('torque_rpm',',',''))
df = df.withColumn('torque_rpm',
                   F.col('torque_rpm').cast('Int'))
df = df.drop('torque')

# # Extract numeric values and RPM for power
# 'power_hp'
df = df.withColumn('power_hp',
                   F.regexp_extract(F.col('power'), hp_pattern, 1).cast('Int'))

# 'power_rpm'
df = df.withColumn('power_rpm',
                   F.regexp_extract('power', rpm_pattern, 1))\
.withColumn('power_rpm',
            F.regexp_replace('power_rpm',',',''))
df = df.withColumn('power_rpm',
                   df.power_rpm.cast('Int'))
df = df.drop('power')

# Salvage
df = df.withColumn('salvage',
                   df.salvage.cast('Boolean'))

# Saving Amount
df = df.withColumn('savings_amount',
                   df.savings_amount.cast('Int'))

# Seller Rating
df = df.withColumn('seller_rating',
                   F.when(F.col('seller_rating').between(1,5),
                          F.round(F.col('seller_rating'),2)).otherwise(None))

# sp_id
df = df.withColumnRenamed('sp_id','dealer_id')
df = df.withColumn('dealer_id',F.col('dealer_id').cast('Int'))
# sp_name
df = df.withColumnRenamed('sp_name',
                          'dealer')

# theft_title
df = df.withColumn('theft_title',
                   df.theft_title.cast('Boolean'))

# even after cleaninig using the price column the data is still very dirty, so I'm using the transmission column
df = df.filter(
        (F.col('transmission')=='M') | (F.col("transmission")=='A')
)

# trim id
df = df.withColumn(
    'trimId', 
    F.when(
        F.col('trimId').rlike('^t'), 
        F.col('trimId')
    ).otherwise(None)
)

# wheel system
df = df.withColumn(
    'wheel_system', 
    F.when(
        F.col('wheel_system').rlike('([A-Z]{3})'), 
        F.col('wheel_system')
    ).otherwise(None)
)

# wheel system display
# depends on the column above
df = df.withColumn(
    'wheel_system_display',
    F.when(
        F.col('wheel_system').isNotNull(),
        F.col('wheel_system_display')
    ).otherwise(None)
)

# Wheel Base (inches)
df = df.withColumn('wheelbase',
                   F.regexp_replace(string='wheelbase',pattern='[^0-9.]',replacement='')) # inches
df = df.withColumn('wheelbase',
                   df.wheelbase.cast('Float'))
df = df.withColumnRenamed('wheelbase',
                          'wheelbase_inches')

# width (inches)
df = df.withColumn('width',
                   F.regexp_replace(string='width',pattern='[^0-9.]',replacement='')) # inches
df = df.withColumn('width',
                   df.width.cast('Float'))
df = df.withColumnRenamed('width',
                          'width_inches')

# year
df = df.withColumn(
    'year', 
    F.when(
        F.col('year').between(1800,2024), 
        F.col('year')
    ).otherwise(None)
)

df = df.withColumnRenamed('year',
                         'year_of_make')

In [16]:
df = df.replace('',None)

# Joining and Merging

### Vehicle Table

In [17]:
# Sub Dimensional Tables from the Vehicle Dimension Table

dim_vehicle_interior = df.select(
    'front_legroom_inches','interior_color','maximum_seating','back_legroom_inches'
).distinct()

dim_vehicle_body = df.select(
    'trimId','trim_name', 'model_name', 'make_name','body_type','wheel_system','wheel_system_display', 'exterior_color', 'listing_color',
    'franchise_make', 'wheelbase_inches', 'height_inches', 'length_inches', 'width_inches'
).distinct()

dim_vehicle_engine = df.select(
    'engine_cylinders', 'engine_displacement', 'engine_type', 'torque_lbft', 'torque_rpm', 'power_hp','power_rpm'
).distinct()

dim_vehicle_fuel = df.select(
    'fuel_tank_volume_gallons', 'fuel_type', 'city_fuel_economy', 'highway_fuel_economy'
).distinct()

dim_vehicle_transmission = df.select(
    'transmission', 'transmission_display'
).distinct()

In [18]:
dim_vehicle_interior.createOrReplaceTempView('dim_vehicle_interior')
dim_vehicle_interior = spark.sql('select row_number() over (order by "some_column") as interior_id, * from dim_vehicle_interior')

dim_vehicle_body.createOrReplaceTempView('dim_vehicle_body')
dim_vehicle_body = spark.sql('select row_number() over (order by "some_column") as body_id, * from dim_vehicle_body')

dim_vehicle_fuel.createOrReplaceTempView('dim_vehicle_fuel')
dim_vehicle_fuel = spark.sql('select row_number() over (order by "some_column") as fuel_id, * from dim_vehicle_fuel')

dim_vehicle_engine.createOrReplaceTempView('dim_vehicle_engine')
dim_vehicle_engine = spark.sql('select row_number() over (order by "some_column") as engine_id, * from dim_vehicle_engine')

dim_vehicle_transmission.createOrReplaceTempView('dim_vehicle_transmission')
dim_vehicle_transmission = spark.sql('select row_number() over (order by "some_column") as transmission_id, * from dim_vehicle_transmission')

In [19]:
dim_vehicle = df.join(
    other=dim_vehicle_interior,
    on=['front_legroom_inches','interior_color','maximum_seating','back_legroom_inches'],
    how='left'
)

dim_vehicle = dim_vehicle.join(
    other=dim_vehicle_body,
    on=['trimId','trim_name', 'model_name', 'make_name','body_type','wheel_system','wheel_system_display', 'exterior_color', 'listing_color', 
        'franchise_make', 'wheelbase_inches', 'height_inches', 'length_inches', 'width_inches'],
    how='left'
)

dim_vehicle = dim_vehicle.join(
    other=dim_vehicle_engine,
    on=['engine_cylinders', 'engine_displacement', 'engine_type', 'torque_lbft', 'torque_rpm', 'power_hp','power_rpm'],
    how='left'
)

dim_vehicle = dim_vehicle.join(
    other=dim_vehicle_fuel,
    on=['fuel_tank_volume_gallons', 'fuel_type', 'city_fuel_economy', 'highway_fuel_economy'],
    how='left'
)

dim_vehicle = dim_vehicle.join(
    other=dim_vehicle_transmission,
    on=['transmission', 'transmission_display'],
    how='left'
)

# dim_vehicle = dim_vehicle.withColumn('vehicle_surrogate_id', F.monotonically_increasing_id())
dim_vehicle.createOrReplaceTempView('dim_vehicle')
dim_vehicle = spark.sql('select row_number() over (order by "some_column") as vehicle_surrogate_id, * from dim_vehicle')

dim_vehicle = dim_vehicle.select('vehicle_surrogate_id', 'vin', 'interior_id', 'body_id', 'engine_id', 'fuel_id', 'transmission_id', 'major_options',
                                 'year_of_make', 'description')

### Date Table

In [20]:
dim_date = df.select(
    'listed_date',
    F.year(F.col('listed_date')).alias('year'),
    F.month(F.col('listed_date')).alias('month'),
    F.day(F.col('listed_date')).alias('day')
).distinct()

# dim_date = dim_date.withColumn('date_id', F.monotonically_increasing_id())
dim_date.createOrReplaceTempView('dim_date')
dim_date = spark.sql('select row_number() over (order by "some_column") as date_id, * from dim_date')

### Dealer Table

In [21]:
dim_dealer = df.select('dealer_id', 'dealer', 'dealer_zip', 'franchise_dealer', 'seller_rating').distinct()

# dim_dealer = dim_dealer.withColumn('dealer_surrogate_id', F.monotonically_increasing_id())
dim_dealer.createOrReplaceTempView('dim_dealer')
dim_dealer = spark.sql('select row_number() over (order by "some_column") as dealer_surrogate_id, * from dim_dealer')

### Location Table

In [22]:
dim_location = df.select('longitude', 'latitude', 'city').distinct()

# dim_location = dim_location.withColumn('location_id', F.monotonically_increasing_id())
dim_location.createOrReplaceTempView('dim_location')
dim_location = spark.sql('select row_number() over (order by "some_column") as location_id, * from dim_location')

### Ownership Table

In [23]:
dim_ownership_status = df.select('owner_count', 'fleet', 'frame_damaged', 'has_accidents', 'isCab', 'is_new', 'is_cpo', 'is_oemcpo', 'salvage',
                                 'theft_title').distinct()

# dim_ownership_status = dim_ownership_status.withColumn('ownership_status_id', F.monotonically_increasing_id())
dim_ownership_status.createOrReplaceTempView('dim_ownership_status')
dim_ownership_status = spark.sql('select row_number() over (order by "some_column") as ownership_status_id, * from dim_ownership_status')

### Fact Listings Table

In [24]:
fact_listings = df.join(
    dim_vehicle,
    ['vin'],
    'left'
)

fact_listings = fact_listings.join(
    dim_date,
    ['listed_date'],
    'left'
)

fact_listings = fact_listings.join(
    dim_dealer,
    ['dealer', 'dealer_zip', 'franchise_dealer', 'seller_rating'],
    'left'
)

fact_listings = fact_listings.join(
    dim_location,
    ['longitude', 'latitude', 'city'],
    'left'
)

fact_listings = fact_listings.join(
    dim_ownership_status,
    ['owner_count', 'fleet', 'frame_damaged', 'has_accidents', 'isCab', 'is_new', 'is_cpo', 'is_oemcpo', 'salvage', 'theft_title'],
    'left'
)

In [25]:
fact_listings = fact_listings.select('listing_id', 'dealer_surrogate_id', 'location_id', 'date_id', 'vehicle_surrogate_id', 'ownership_status_id', 
                                     'savings_amount', 'mileage', 'daysonmarket', 'price')

fact_listings.createOrReplaceTempView('fact_listings')
fact_listings = spark.sql('select row_number() over (order by "some_column") as listing_surrogate_id, * from fact_listings')

# Data Loading

In [26]:
mysql_url = 'jdbc:mysql://localhost:3306/cars'
user = 'root'
password = 'anon'

In [27]:
# Vehicle Sub-Dimensional Tables

dim_vehicle_interior.select(
    'interior_id', 'front_legroom_inches','interior_color','maximum_seating','back_legroom_inches').write \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", mysql_url) \
    .option("dbtable", "dim_vehicle_interior") \
    .option("user", user) \
    .option("password", password) \
    .mode("append") \
    .save()

dim_vehicle_body.select(
    'body_id', 'trimId','trim_name', 'model_name', 'make_name','body_type','wheel_system','wheel_system_display', 'exterior_color', 'listing_color',
    'franchise_make', 'wheelbase_inches', 'height_inches', 'length_inches', 'width_inches').write \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", mysql_url) \
    .option("dbtable", "dim_vehicle_body") \
    .option("user", user) \
    .option("password", password) \
    .mode("append") \
    .save()

dim_vehicle_engine.select(
    'engine_id', 'engine_cylinders', 'engine_displacement', 'engine_type', 'torque_lbft', 'torque_rpm', 'power_hp','power_rpm').write \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", mysql_url) \
    .option("dbtable", "dim_vehicle_engine") \
    .option("user", user) \
    .option("password", password) \
    .mode("append") \
    .save()

dim_vehicle_transmission.select(
    'transmission_id', 'transmission', 'transmission_display').write \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", mysql_url) \
    .option("dbtable", "dim_vehicle_transmission") \
    .option("user", user) \
    .option("password", password) \
    .mode("append") \
    .save()

dim_vehicle_fuel.select(
    'fuel_id', 'fuel_tank_volume_gallons', 'fuel_type', 'city_fuel_economy', 'highway_fuel_economy').write \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", mysql_url) \
    .option("dbtable", "dim_vehicle_fuel") \
    .option("user", user) \
    .option("password", password) \
    .mode("append") \
    .save()

In [28]:
dim_vehicle.select(
    'vehicle_surrogate_id', 'vin', 'interior_id', 'body_id', 'engine_id', 'fuel_id', 'transmission_id', 'major_options', 'year_of_make', 
    'description').write \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", mysql_url) \
    .option("dbtable", "dim_vehicle") \
    .option("user", user) \
    .option("password", password) \
    .mode("append") \
    .save()

dim_date.select(
    'date_id', 'listed_date', 'year', 'month', 'day').write \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", mysql_url) \
    .option("dbtable", 'dim_date') \
    .option("user", user) \
    .option("password", password) \
    .mode("append") \
    .save()

dim_dealer.select(
     'dealer_surrogate_id', 'dealer_id', 'dealer', 'dealer_zip', 'franchise_dealer', 'seller_rating').write \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", mysql_url) \
    .option("dbtable", "dim_dealer") \
    .option("user", user) \
    .option("password", password) \
    .mode("append") \
    .save()

dim_location.select(
     'location_id', 'longitude', 'latitude', 'city').write \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", mysql_url) \
    .option("dbtable", "dim_location") \
    .option("user", user) \
    .option("password", password) \
    .mode("append") \
    .save()

dim_ownership_status.select(
     'ownership_status_id', 'owner_count', 'fleet', 'frame_damaged', 'has_accidents', 'isCab', 'is_cpo', 'is_oemcpo', 'salvage', 'theft_title').write \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", mysql_url) \
    .option("dbtable", "dim_ownership_status") \
    .option("user", user) \
    .option("password", password) \
    .mode("append") \
    .save()

In [29]:
fact_listings.select(
    'listing_surrogate_id', 'listing_id', 'dealer_surrogate_id', 'location_id', 'date_id', 'vehicle_surrogate_id', 'ownership_status_id',
    'savings_amount', 'mileage', 'daysonmarket','price').write \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", mysql_url) \
    .option("dbtable", "fact_listings") \
    .option("user", user) \
    .option("password", password) \
    .mode("append") \
    .save()